In [88]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import MatrixBasedConvolution as mbc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
inputs_shape=(1,5,5,1)
ARGS={"kernel_size":3,
"strides":1,
"padding":'same',
"use_phi":True,
"activation":None,
"use_lambda_out":False,
"use_lambda_in":False}

In [90]:
con2d=mbc.matrix_conv_2d(**ARGS)
con2d.build(inputs_shape)

In [91]:
inputs=tf.random.uniform(shape=inputs_shape,minval=0,maxval=1)


In [99]:
ouputs_spectral=con2d.conv_jit(inputs)[0,:,:,0].numpy()
ouputs_spectral.shape

TypeError: in user code:

    File "c:\Users\jketchak\Documents\DALPHAcommunity\UNAMUR\EUREKA\btwg_ml\MatrixBasedConvolution-mbc\MatrixBasedConvolution\mbc.py", line 272, in indices_phi  *
        if i == count * (self.output_shape[1]):

    TypeError: Input 'y' of 'Mul' Op has type float32 that does not match type int32 of argument 'x'.


In [93]:
kernel=tf.reshape(con2d.kernel,shape=(ARGS.get('kernel_size'),ARGS.get('kernel_size')))
kernel.shape

TensorShape([3, 3])

In [94]:
ouputs_classical=mbc.classical_convolution(img=inputs[0,:,:,0].numpy(),kernel=kernel.numpy())
ouputs_classical.shape


(5, 5)

In [95]:
ouputs_spectral

array([[-0.92997015, -1.1436429 , -1.2366762 , -1.0699443 , -0.60976756],
       [-0.8111584 , -1.1342181 , -1.0453719 , -1.3929131 , -0.7784873 ],
       [-0.64596725, -1.2239282 , -1.7971048 , -1.434051  , -0.76620597],
       [-0.8003308 , -1.7845265 , -1.1509647 , -1.46482   , -0.5464281 ],
       [-0.41989923, -0.6421689 , -1.0164733 , -0.65374154, -0.26405677]],
      dtype=float32)

In [96]:
ouputs_classical

array([[-0.92997015, -1.1436428 , -1.2366761 , -1.0699443 , -0.6097675 ],
       [-0.81115854, -1.134218  , -1.0453718 , -1.3929131 , -0.7784873 ],
       [-0.64596725, -1.2239283 , -1.797105  , -1.434051  , -0.76620597],
       [-0.8003308 , -1.7845265 , -1.1509647 , -1.4648199 , -0.5464281 ],
       [-0.41989923, -0.6421689 , -1.0164733 , -0.6537416 , -0.26405674]],
      dtype=float32)

In [97]:
if np.allclose(ouputs_classical,ouputs_spectral,atol=1e-16):
    print("The outputs are the same")
    

The outputs are the same
